In [0]:
import pandas as pd

df_train = pd.read_csv('train.tsv',sep='\t')

In [0]:
df_test = pd.read_csv('dev.tsv',sep='\t')

In [0]:
df_train.head()

,Unnamed: 0,ids,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source,Concepts
0,0,LIPITOR.449.txt,Extreme tiredness and flatulence. Not sure whe...,0,1,0,0,0,flatulence|exhaustion|tired|Not sure|Extreme|t...
1,1,LIPITOR.188.txt,1/7/05-continued. not all of it posted before....,0,1,0,1,0,package insert|depression|Lipitor|cholesterol|...
2,2,LIPITOR.541.txt,So sad to see so many with problems like mine!...,1,1,1,0,0,muscle pain|joint pain|depression|Lipitor|Lipi...
3,3,LIPITOR.810.txt,Within 1 month time developed severe depressio...,0,1,0,0,0,severe depression|headaches|Lipitor|statins|Li...
4,4,LIPITOR.393.txt,I have been on lipitor for 10 years for heart ...,0,1,0,0,0,leg weakness|changed|experience|Potassium|cram...


In [0]:
df_test.head()

,Unnamed: 0,ids,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source,Concepts
0,0,LIPITOR.595.txt,"Swelling left arm, very severe itching, intole...",0,1,0,0,0,Swelling|itching|left arm|hand|dreams|bruise|v...
1,1,ARTHROTEC.101.txt,"1st pill taken with food, a few hours after i ...",0,1,0,0,0,side effects|depression|cramping|upset|experie...
2,2,LIPITOR.367.txt,episode of intense dizziness lasting nearly an...,0,1,0,0,0,dizziness|lassitude|chills|shivers|problem|wor...
3,3,LIPITOR.74.txt,After taking Crestor and having muscle pain an...,0,1,0,0,1,liver problems|decreased|itchy|control|crawly|...
4,4,LIPITOR.389.txt,"75 yo mother-in-law has memory loss, hair loss...",0,1,0,1,0,lack of appetite|hair loss|stroke|sciatica|los...


In [0]:
train_body =df_train['body'].to_list()
test_body =df_test['body'].to_list()

In [0]:
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

for idx,text in enumerate(train_body):
  train_body[idx] = clean_text(text)

for idx,text in enumerate(test_body):
  test_body[idx] = clean_text(text)

In [0]:
from collections import Counter

str_total = ' '.join(train_body) + ' '.join(test_body)
list_total = str_total.split()
dict_count = Counter(list_total)

vocab = dict()
for key,val in dict_count.items():
  vocab[key] = len(vocab)

In [0]:
print(len(vocab))

6640


In [0]:
x_train = list()
x_test = list()
def encode_text(toks,type='train'):
  encod_arr = [None]*len(toks)
  try:
    for idx,tok in enumerate(toks):
      encod_arr[idx] = vocab[tok]
  except: pass

  if type =='train' : x_train.append(encod_arr)
  if type =='test' : x_test.append(encod_arr)


for item in train_body:
  encode_text(item.split(), 'train')

for item in test_body:
  encode_text(item.split(), 'test')

In [0]:
print(x_train[0],train_body[0])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 9, 17, 18, 19, 8, 7, 20, 8, 7, 21, 22, 23, 24, 8, 7, 9, 10] extreme tiredness and flatulence not sure whether it is the lipitor but i am now tired to the point of exhaustion is it work is it me getting older or is it the lipitor


# **FastText Deep Learning Model**

In [0]:
import numpy as np
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras import Input, Model
from keras.layers import Embedding, GlobalAveragePooling1D, Dense
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)

In [0]:
class FastText(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = GlobalAveragePooling1D()(embedding)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [0]:
def create_ngram_set(input_list, ngram_value=2):
    """
    Extract a set of n-grams from a list of integers.
    # >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=2)
    {(4, 9), (4, 1), (1, 4), (9, 4)}
    # >>> create_ngram_set([1, 4, 9, 4, 1, 4], ngram_value=3)
    [(1, 4, 9), (4, 9, 4), (9, 4, 1), (4, 1, 4)]
    """
    return set(zip(*[input_list[i:] for i in range(ngram_value)]))


def add_ngram(sequences, token_indice, ngram_range=2):
    """
    Augment the input list of list (sequences) by appending n-grams values.
    Example: adding bi-gram
    # >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    # >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017}
    # >>> add_ngram(sequences, token
 [0.04929424 0.9507057 ]_indice, ngram_range=2)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42]]
    Example: adding tri-gram
    # >>> sequences = [[1, 3, 4, 5], [1, 3, 7, 9, 2]]
    # >>> token_indice = {(1, 3): 1337, (9, 2): 42, (4, 5): 2017, (7, 9, 2): 2018}
    # >>> add_ngram(sequences, token_indice, ngram_range=3)
    [[1, 3, 4, 5, 1337, 2017], [1, 3, 7, 9, 2, 1337, 42, 2018]]
    """
    new_sequences = []
    for input_list in sequences:
        new_list = input_list[:]
        for ngram_value in range(2, ngram_range + 1):
            for i in range(len(new_list) - ngram_value + 1):
                ngram = tuple(new_list[i:i + ngram_value])
                if ngram in token_indice:
                    new_list.append(token_indice[ngram])
        new_sequences.append(new_list)

    return new_sequences


# Set parameters:
# ngram_range = 2 will add bi-grams features
ngram_range = 1
max_features = len(vocab)
maxlen = max([len(item) for item in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

if ngram_range > 1:
    print('Adding {}-gram features'.format(ngram_range))
    # Create set of unique n-gram from the training set.
    ngram_set = set()
    for input_list in x_train:
        for i in range(2, ngram_range + 1):
            set_of_ngram = create_ngram_set(input_list, ngram_value=i)
            ngram_set.update(set_of_ngram)

    # Dictionary mapping n-gram token to a unique integer.
    # Integer values are greater than max_features in order
    # to avoid collision with existing features.
    start_index = max_features + 1
    token_indice = {v: k + start_index for k, v in enumerate(ngram_set)}
    indice_token = {token_indice[k]: k for k in token_indice}

    # max_features is the highest integer that could be found in the dataset.
    max_features = np.max(list(indice_token.keys())) + 1

    # Augmenting x_train and x_test with n-grams features
    x_train = add_ngram(x_train, token_indice, ngram_range)
    x_test = add_ngram(x_test, token_indice, ngram_range)
    print('Average train sequence length: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = FastText(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'mean_squared_error', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')

categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']

for category in categories:
    print('Reporting Category: ', category)
    y_test =  np.zeros((df_test.shape[0],2),dtype=int)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()


    for rowid in range(y_test.shape[0]):
      y_test[rowid, int(y_test_label[rowid])] = 1

    x_train_res, y_train_res = sm.fit_resample(x_train,y_train_label)

    y_train =  np.zeros((y_train_res.shape[0],2),dtype=int)
    for rowid in range(y_train_res.shape[0]):
      y_train[rowid, int(y_train_res[rowid])] = 1


    model.fit(x_train_res,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))



Loading data...
942 train sequences
300 test sequences
Average train sequence length: 86
Average test sequence length: 80
Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Train on 1550 samples, validate on 300 samples
Epoch 1/10
1550/1550 [==============================] - 0s 277us/step - loss: 0.2477 - acc: 0.5348 - val_loss: 0.2341 - val_acc: 0.8567
Epoch 2/10
1550/1550 [==============================] - 0s 132us/step - loss: 0.2415 - acc: 0.6865 - val_loss: 0.2284 - val_acc: 0.8567
Epoch 3/10
1550/1550 [==============================] - 0s 151us/step - loss: 0.2324 - acc: 0.7277 - val_loss: 0.2198 - val_acc: 0.8567
Epoch 4/10
1550/1550 [==============================] - 0s 135us/step - loss: 0.2191 - acc: 0.7723 - val_loss: 0.2120 - val_acc: 0.8567
Test...
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       257
           1       0.00      0.00      0.00        43

    accuracy                           0.86       300
   macro avg       0.43      0.50      0.46       300
weighted avg       0.73      0.86      0.79       300

Reporting Category:  Results and Side-Effects Observed
Train on 1866 sampl

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1866/1866 [==============================] - 0s 151us/step - loss: 0.2735 - acc: 0.3451 - val_loss: 0.2615 - val_acc: 0.1967
Epoch 2/10
1866/1866 [==============================] - 0s 144us/step - loss: 0.2560 - acc: 0.5000 - val_loss: 0.2554 - val_acc: 0.3767
Epoch 3/10
1866/1866 [==============================] - 0s 137us/step - loss: 0.2452 - acc: 0.5536 - val_loss: 0.2559 - val_acc: 0.3933
Epoch 4/10
1866/1866 [==============================] - 0s 121us/step - loss: 0.2356 - acc: 0.6399 - val_loss: 0.2514 - val_acc: 0.4500
Epoch 5/10
1866/1866 [==============================] - 0s 118us/step - loss: 0.2234 - acc: 0.6811 - val_loss: 0.2416 - val_acc: 0.4900
Epoch 6/10
1866/1866 [==============================] - 0s 115us/step - loss: 0.2079 - acc: 0.7444 - val_loss: 0.2125 - val_acc: 0.6067
Epoch 7/10
1866/1866 [==============================] - 0s 138us/step - loss: 0.1887 - acc: 0.8039 - val_loss: 0.2026 - val_acc: 0.6200
Epoch 8/10
1866/1866 [==============================] - 0s 

...

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1668/1668 [==============================] - 0s 153us/step - loss: 0.1822 - acc: 0.7428 - val_loss: 0.1697 - val_acc: 0.8267
Epoch 2/10
1668/1668 [==============================] - 0s 156us/step - loss: 0.1607 - acc: 0.8028 - val_loss: 0.1544 - val_acc: 0.8667
Epoch 3/10
1668/1668 [==============================] - 0s 158us/step - loss: 0.1427 - acc: 0.8447 - val_loss: 0.1564 - val_acc: 0.8700
Epoch 4/10
1668/1668 [==============================] - 0s 154us/step - loss: 0.1275 - acc: 0.8897 - val_loss: 0.1348 - val_acc: 0.8733
Epoch 5/10
1668/1668 [==============================] - 0s 156us/step - loss: 0.1149 - acc: 0.8981 - val_loss: 0.1367 - val_acc: 0.8767
Epoch 6/10
1668/1668 [==============================] - 0s 127us/step - loss: 0.1059 - acc: 0.9089 - val_loss: 0.1297 - val_acc: 0.8800
Epoch 7/10
1668/1668 [==============================] - 0s 162us/step - loss: 0.0980 - acc: 0.9197 - val_loss: 0.1259 - val_acc: 0.8800
Epoch 8/10
1668/1668 [==============================] - 0s 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1756/1756 [==============================] - 0s 122us/step - loss: 0.0772 - acc: 0.9174 - val_loss: 0.1000 - val_acc: 0.9200
Epoch 2/10
1756/1756 [==============================] - 0s 118us/step - loss: 0.0729 - acc: 0.9231 - val_loss: 0.0932 - val_acc: 0.9200
Epoch 3/10
1756/1756 [==============================] - 0s 131us/step - loss: 0.0698 - acc: 0.9265 - val_loss: 0.0953 - val_acc: 0.9200
Epoch 4/10
1756/1756 [==============================] - 0s 123us/step - loss: 0.0666 - acc: 0.9311 - val_loss: 0.1006 - val_acc: 0.9200
Test...
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       276
           1       0.00      0.00      0.00        24

    accuracy                           0.92       300
   macro avg       0.46      0.50      0.48       300
weighted avg       0.85      0.92      0.88       300



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
y_train_res.shape[0]

1550

#**TextCNN Deep Learning Model**

In [0]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout


class TextCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        # Embedding part can try multichannel as same as origin paper
        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        convs = []
        for kernel_size in [3, 4, 5]:
            c = Conv1D(128, kernel_size, activation='relu')(embedding)
            c = GlobalMaxPooling1D()(c)
            convs.append(c)
        x = Concatenate()(convs)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model


In [0]:
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.utils import to_categorical

ngram_range = 1
max_features = len(vocab)
maxlen = max([len(item) for item in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')

for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    x_train_res, y_train_res = sm.fit_resample(x_train,y_train_label)

    y_train =  to_categorical(y_train_res)
    y_test =   to_categorical(y_test_label)

    model.fit(x_train_res,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))


Loading data...
942 train sequences
300 test sequences
Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1550 samples, validate on 300 samples
Epoch 1/10
1550/1550 [==============================] - 6s 4ms/step - loss: 0.6085 - acc: 0.7716 - val_loss: 0.4814 - val_acc: 0.8533
Epoch 2/10
1550/1550 [==============================] - 1s 332us/step - loss: 0.3210 - acc: 0.8787 - val_loss: 0.4395 - val_acc: 0.8367
Epoch 3/10
1550/1550 [==============================] - 1s 337us/step - loss: 0.2108 - acc: 0.9065 - val_loss: 0.4019 - val_acc: 0.8567
Epoch 4/10
1550/1550 [==============================] - 1s 354us/step - loss: 0.1285 - acc: 0.9561 - val_loss: 0.4575 - val_acc: 0.7733
Epoch 5/10
1550/1550 [==============================] - 1s 348us/step - loss: 0.0712 - acc: 0.9781 - val_loss: 0.4541 - val_acc: 0.8100
Epoch 6/10
1550/1550 [==============================] - 1s 342us/step - loss: 0.0316 - acc: 0.9942 - val_loss: 0.4984 - val_acc: 0.8167
Test...
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       257
           1       0.2

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1866/1866 [==============================] - 1s 362us/step - loss: 0.9741 - acc: 0.6977 - val_loss: 0.2221 - val_acc: 0.9167
Epoch 2/10
1866/1866 [==============================] - 1s 333us/step - loss: 0.0884 - acc: 0.9743 - val_loss: 0.1183 - val_acc: 0.9633
Epoch 3/10
1866/1866 [==============================] - 1s 333us/step - loss: 0.0408 - acc: 0.9887 - val_loss: 0.1140 - val_acc: 0.9667
Epoch 4/10
1866/1866 [==============================] - 1s 331us/step - loss: 0.0256 - acc: 0.9941 - val_loss: 0.1050 - val_acc: 0.9733
Epoch 5/10
1866/1866 [==============================] - 1s 323us/step - loss: 0.0172 - acc: 0.9973 - val_loss: 0.1048 - val_acc: 0.9767
Epoch 6/10
1866/1866 [==============================] - 1s 339us/step - loss: 0.0126 - acc: 0.9984 - val_loss: 0.1017 - val_acc: 0.9767
Epoch 7/10
1866/1866 [==============================] - 1s 330us/step - loss: 0.0095 - acc: 0.9984 - val_loss: 0.0968 - val_acc: 0.9733
Epoch 8/10
1866/1866 [==============================] - 1s 

...

1484/1484 [==============================] - 1s 352us/step - loss: 1.3619 - acc: 0.5202 - val_loss: 0.6202 - val_acc: 0.6833
Epoch 2/10
1484/1484 [==============================] - 0s 334us/step - loss: 0.3510 - acc: 0.8565 - val_loss: 0.4664 - val_acc: 0.7933
Epoch 3/10
1484/1484 [==============================] - 0s 335us/step - loss: 0.2100 - acc: 0.9178 - val_loss: 0.4366 - val_acc: 0.7967
Epoch 4/10
1484/1484 [==============================] - 1s 345us/step - loss: 0.1239 - acc: 0.9690 - val_loss: 0.4483 - val_acc: 0.7800
Epoch 5/10
1484/1484 [==============================] - 1s 346us/step - loss: 0.0694 - acc: 0.9879 - val_loss: 0.4758 - val_acc: 0.7867
Epoch 6/10
1484/1484 [==============================] - 0s 330us/step - loss: 0.0372 - acc: 0.9946 - val_loss: 0.4964 - val_acc: 0.8100
Epoch 7/10
1484/1484 [==============================] - 0s 335us/step - loss: 0.0216 - acc: 0.9987 - val_loss: 0.5328 - val_acc: 0.7967
Epoch 8/10
1484/1484 [==============================] - 1s 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1668/1668 [==============================] - 1s 374us/step - loss: 0.4496 - acc: 0.8735 - val_loss: 0.3656 - val_acc: 0.8800
Epoch 2/10
1668/1668 [==============================] - 1s 337us/step - loss: 0.1571 - acc: 0.9406 - val_loss: 0.3267 - val_acc: 0.8933
Epoch 3/10
1668/1668 [==============================] - 1s 332us/step - loss: 0.0972 - acc: 0.9658 - val_loss: 0.3244 - val_acc: 0.8933
Epoch 4/10
1668/1668 [==============================] - 1s 328us/step - loss: 0.0602 - acc: 0.9808 - val_loss: 0.3089 - val_acc: 0.8867
Epoch 5/10
1668/1668 [==============================] - 1s 331us/step - loss: 0.0326 - acc: 0.9946 - val_loss: 0.3426 - val_acc: 0.8933
Epoch 6/10
1668/1668 [==============================] - 1s 333us/step - loss: 0.0191 - acc: 0.9982 - val_loss: 0.3309 - val_acc: 0.8900
Test...
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       264
           1       0.57      0.36      0.44        36

    accuracy           

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


1756/1756 [==============================] - 1s 377us/step - loss: 0.2759 - acc: 0.9174 - val_loss: 0.2848 - val_acc: 0.9200
Epoch 2/10
1756/1756 [==============================] - 1s 336us/step - loss: 0.0946 - acc: 0.9692 - val_loss: 0.2255 - val_acc: 0.9133
Epoch 3/10
1756/1756 [==============================] - 1s 332us/step - loss: 0.0561 - acc: 0.9778 - val_loss: 0.2255 - val_acc: 0.9200
Epoch 4/10
1756/1756 [==============================] - 1s 332us/step - loss: 0.0298 - acc: 0.9920 - val_loss: 0.2180 - val_acc: 0.9133
Epoch 5/10
1756/1756 [==============================] - 1s 355us/step - loss: 0.0173 - acc: 0.9983 - val_loss: 0.2546 - val_acc: 0.9167
Epoch 6/10
1756/1756 [==============================] - 1s 335us/step - loss: 0.0106 - acc: 1.0000 - val_loss: 0.2494 - val_acc: 0.9167
Test...
              precision    recall  f1-score   support

           0       0.93      0.99      0.96       276
           1       0.40      0.08      0.14        24

    accuracy           

#**RCNN**


In [0]:
from keras import Input, Model
from keras import backend as K
from keras.layers import Embedding, Dense, SimpleRNN, Lambda, Concatenate, Conv1D, GlobalMaxPooling1D


class RCNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input_current = Input((self.maxlen,))
        input_left = Input((self.maxlen,))
        input_right = Input((self.maxlen,))

        embedder = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)
        embedding_current = embedder(input_current)
        embedding_left = embedder(input_left)
        embedding_right = embedder(input_right)

        x_left = SimpleRNN(128, return_sequences=True)(embedding_left)
        x_right = SimpleRNN(128, return_sequences=True, go_backwards=True)(embedding_right)
        x_right = Lambda(lambda x: K.reverse(x, axes=1))(x_right)
        x = Concatenate(axis=2)([x_left, embedding_current, x_right])

        x = Conv1D(64, kernel_size=1, activation='tanh')(x)
        x = GlobalMaxPooling1D()(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=[input_current, input_left, input_right], outputs=output)
        return model

In [0]:
import numpy as np
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = len(vocab)
maxlen = max([len(x) for x in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Prepare input for model...')
x_train_current = x_train
x_train_left = np.hstack([np.expand_dims(x_train[:, 0], axis=1), x_train[:, 0:-1]])
x_train_right = np.hstack([x_train[:, 1:], np.expand_dims(x_train[:, -1], axis=1)])
x_test_current = x_test
x_test_left = np.hstack([np.expand_dims(x_test[:, 0], axis=1), x_test[:, 0:-1]])
x_test_right = np.hstack([x_test[:, 1:], np.expand_dims(x_test[:, -1], axis=1)])
print('x_train_current shape:', x_train_current.shape)
print('x_train_left shape:', x_train_left.shape)
print('x_train_right shape:', x_train_right.shape)
print('x_test_current shape:', x_test_current.shape)
print('x_test_left shape:', x_test_left.shape)
print('x_test_right shape:', x_test_right.shape)

print('Build model...')
model = RCNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])


categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']

for category in categories:
    print('Reporting Category: ', category)
    y_train =  np.zeros((df_train.shape[0],2),dtype=int)
    y_test =  np.zeros((df_test.shape[0],2),dtype=int)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    for rowid in range(y_train.shape[0]):
      y_train[rowid, int(y_train_label[rowid])] = 1

    for rowid in range(y_test.shape[0]):
      y_test[rowid, int(y_test_label[rowid])] = 1

    model.fit([x_train_current, x_train_left, x_train_right], y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=([x_test_current, x_test_left, x_test_right], y_test))
    print('Test...')
    result = model.predict([x_test_current, x_test_left, x_test_right])
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))

942 train sequences
300 test sequences
Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Prepare input for model...
x_train_current shape: (942, 707)
x_train_left shape: (942, 707)
x_train_right shape: (942, 707)
x_test_current shape: (300, 707)
x_test_left shape: (300, 707)
x_test_right shape: (300, 707)
Build model...
Reporting Category:  Uncertainity of Post_Diagnosis
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 18s 19ms/step - loss: 0.4901 - acc: 0.8227 - val_loss: 0.4192 - val_acc: 0.8567
Epoch 2/10
942/942 [==============================] - 17s 18ms/step - loss: 0.4678 - acc: 0.8227 - val_loss: 0.4123 - val_acc: 0.8567
Epoch 3/10
942/942 [==============================] - 17s 18ms/step - loss: 0.4633 - acc: 0.8227 - val_loss: 0.4086 - val_acc: 0.8567
Epoch 4/10
942/942 [==============================] - 17s 18ms/step - loss: 0.4598 - acc: 0.8227 - val_loss: 0.4599 - val_acc: 0.8567
Epoch 5/1

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.86      1.00      0.92       257
           1       0.00      0.00      0.00        43

    accuracy                           0.86       300
   macro avg       0.43      0.50      0.46       300
weighted avg       0.73      0.86      0.79       300

Reporting Category:  Results and Side-Effects Observed
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 17s 18ms/step - loss: 0.2365 - acc: 0.8811 - val_loss: 0.0564 - val_acc: 0.9900
Epoch 2/10
942/942 [==============================] - 17s 18ms/step - loss: 0.0548 - acc: 0.9904 - val_loss: 0.0562 - val_acc: 0.9900
Epoch 3/10
942/942 [==============================] - 17s 18ms/step - loss: 0.0541 - acc: 0.9904 - val_loss: 0.0560 - val_acc: 0.9900
Epoch 4/10
942/942 [==============================] - 17s 18ms/step - loss: 0.0541 - acc: 0.9904 - val_loss: 0.0560 - val_acc: 0.9900
Epoch 5/10
942/942 [================

#**TextAttBiRNN**


In [0]:
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [0]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, Bidirectional, CuDNNLSTM

class TextAttBiRNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedding)  # LSTM or GRU
        x = Attention(self.maxlen)(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model


In [0]:
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import numpy as np
from sklearn.metrics import classification_report

max_features = len(vocab)
maxlen = max([len(item) for item in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print('Loading data...')
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextAttBiRNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']

for category in categories:
    print('Reporting Category: ', category)
    y_train =  np.zeros((df_train.shape[0],2),dtype=int)
    y_test =  np.zeros((df_test.shape[0],2),dtype=int)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    for rowid in range(y_train.shape[0]):
      y_train[rowid, int(y_train_label[rowid])] = 1

    for rowid in range(y_test.shape[0]):
      y_test[rowid, int(y_test_label[rowid])] = 1

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))


Loading data...
942 train sequences
300 test sequences
Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 3s 3ms/step - loss: 0.4933 - acc: 0.8227 - val_loss: 0.4025 - val_acc: 0.8567
Epoch 2/10
942/942 [==============================] - 2s 2ms/step - loss: 0.4462 - acc: 0.8227 - val_loss: 0.4004 - val_acc: 0.8567
Epoch 3/10
942/942 [==============================] - 2s 2ms/step - loss: 0.4102 - acc: 0.8206 - val_loss: 0.3777 - val_acc: 0.8567
Epoch 4/10
942/942 [==============================] - 2s 2ms/step - loss: 0.3493 - acc: 0.8195 - val_loss: 0.4316 - val_acc: 0.7667
Test...
              precision    recall  f1-score   support

           0       0.86      0.87      0.86       257
           1       0.15      0.14      0.15        43

    accuracy                          

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


942/942 [==============================] - 2s 2ms/step - loss: 0.9944 - acc: 0.6582 - val_loss: 0.4858 - val_acc: 0.8100
Epoch 2/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5179 - acc: 0.7877 - val_loss: 0.4874 - val_acc: 0.8100
Epoch 3/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5167 - acc: 0.7877 - val_loss: 0.4899 - val_acc: 0.8100
Epoch 4/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5194 - acc: 0.7877 - val_loss: 0.4915 - val_acc: 0.8100
Test...
              precision    recall  f1-score   support

           0       0.81      1.00      0.90       243
           1       0.00      0.00      0.00        57

    accuracy                           0.81       300
   macro avg       0.41      0.50      0.45       300
weighted avg       0.66      0.81      0.72       300

Reporting Category:  Diet and Maintenance
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 2s 2ms

#**TextBiRNN**

In [0]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, Bidirectional, CuDNNLSTM


class TextBiRNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = Bidirectional(CuDNNLSTM(128))(embedding)  # LSTM or GRU

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model


In [0]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.utils import to_categorical

max_features = len(vocab)
maxlen = max([len(item) for item in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextBiRNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']

for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    y_train = to_categorical(y_train_label, num_classes=None)
    y_test = to_categorical(y_test_label, num_classes=None)

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))

Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 3s 3ms/step - loss: 0.5177 - acc: 0.8036 - val_loss: 0.4064 - val_acc: 0.8567
Epoch 2/10
942/942 [==============================] - 2s 2ms/step - loss: 0.4493 - acc: 0.8227 - val_loss: 0.4043 - val_acc: 0.8567
Epoch 3/10
942/942 [==============================] - 2s 2ms/step - loss: 0.3796 - acc: 0.8227 - val_loss: 0.4158 - val_acc: 0.8433
Epoch 4/10
942/942 [==============================] - 2s 2ms/step - loss: 0.2065 - acc: 0.9098 - val_loss: 0.4674 - val_acc: 0.8267
Test...
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       257
           1       0.24      0.09      0.13        43

    accuracy                           0.83       300
   macro avg       0.55      0.52      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


942/942 [==============================] - 2s 2ms/step - loss: 0.9816 - acc: 0.6624 - val_loss: 0.5301 - val_acc: 0.8100
Epoch 2/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5165 - acc: 0.7877 - val_loss: 0.4870 - val_acc: 0.8100
Epoch 3/10
942/942 [==============================] - 2s 2ms/step - loss: 0.5006 - acc: 0.7877 - val_loss: 0.4873 - val_acc: 0.8100
Epoch 4/10
942/942 [==============================] - 2s 2ms/step - loss: 0.4835 - acc: 0.7877 - val_loss: 0.4945 - val_acc: 0.8100
Test...
              precision    recall  f1-score   support

           0       0.81      1.00      0.90       243
           1       0.00      0.00      0.00        57

    accuracy                           0.81       300
   macro avg       0.41      0.50      0.45       300
weighted avg       0.66      0.81      0.72       300

Reporting Category:  Diet and Maintenance
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 2s 2ms

#**RCNNVariant**

In [0]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Concatenate, Conv1D, Bidirectional, CuDNNLSTM, GlobalAveragePooling1D, GlobalMaxPooling1D


class RCNNVariant(object):
    """Variant of RCNN.

        Base on structure of RCNN, we do some improvement:
        1. Ignore the shift for left/right context.
        2. Use Bidirectional LSTM/GRU to encode context.
        3. Use Multi-CNN to represent the semantic vectors.
        4. Use ReLU instead of Tanh.
        5. Use both AveragePooling and MaxPooling.
    """

    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)

        x_context = Bidirectional(CuDNNLSTM(128, return_sequences=True))(embedding)
        x = Concatenate()([embedding, x_context])

        convs = []
        for kernel_size in range(1, 5):
            conv = Conv1D(128, kernel_size, activation='relu')(x)
            convs.append(conv)
        poolings = [GlobalAveragePooling1D()(conv) for conv in convs] + [GlobalMaxPooling1D()(conv) for conv in convs]
        x = Concatenate()(poolings)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

In [0]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = len(vocab)
maxlen = max([len(item) for item in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = RCNNVariant(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')

for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    y_train = to_categorical(y_train_label, num_classes=None)
    y_test = to_categorical(y_test_label, num_classes=None)

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))

Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 5s 5ms/step - loss: 0.4852 - acc: 0.8227 - val_loss: 0.4197 - val_acc: 0.8567
Epoch 2/10
942/942 [==============================] - 3s 3ms/step - loss: 0.4541 - acc: 0.8227 - val_loss: 0.4038 - val_acc: 0.8567
Epoch 3/10
942/942 [==============================] - 3s 3ms/step - loss: 0.4285 - acc: 0.8238 - val_loss: 0.3906 - val_acc: 0.8567
Epoch 4/10
942/942 [==============================] - 3s 3ms/step - loss: 0.3157 - acc: 0.8503 - val_loss: 0.5010 - val_acc: 0.7367
Test...
              precision    recall  f1-score   support

           0       0.88      0.80      0.84       257
           1       0.24      0.37      0.29        43

    accuracy                           0.74       300
   macro avg       0.56      0.58      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


942/942 [==============================] - 3s 3ms/step - loss: 12.6960 - acc: 0.2123 - val_loss: 13.0557 - val_acc: 0.1900
Epoch 2/10
942/942 [==============================] - 3s 3ms/step - loss: 12.6960 - acc: 0.2123 - val_loss: 13.0557 - val_acc: 0.1900
Epoch 3/10
942/942 [==============================] - 3s 3ms/step - loss: 12.6960 - acc: 0.2123 - val_loss: 13.0557 - val_acc: 0.1900
Epoch 4/10
942/942 [==============================] - 3s 3ms/step - loss: 12.6960 - acc: 0.2123 - val_loss: 13.0557 - val_acc: 0.1900
Test...
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       243
           1       0.19      1.00      0.32        57

    accuracy                           0.19       300
   macro avg       0.10      0.50      0.16       300
weighted avg       0.04      0.19      0.06       300

Reporting Category:  Diet and Maintenance
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] 

#**TextRNN**

In [0]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Dropout, CuDNNLSTM


class TextRNN(object):
    def __init__(self, maxlen, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen = maxlen
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))

        embedding = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen)(input)
        x = CuDNNLSTM(128)(embedding)  # LSTM or GRU

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model


In [0]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = len(vocab)
maxlen = max([len(item) for item in (x_train+x_test)])
batch_size = 32
embedding_dims = 50
epochs = 10

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = TextRNN(maxlen, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')

for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    y_train = to_categorical(y_train_label, num_classes=None)
    y_test = to_categorical(y_test_label, num_classes=None)

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))

Pad sequences (samples x time)...
x_train shape: (942, 707)
x_test shape: (300, 707)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 3s 3ms/step - loss: 0.5269 - acc: 0.8142 - val_loss: 0.3989 - val_acc: 0.8567
Epoch 2/10
942/942 [==============================] - 1s 1ms/step - loss: 0.4392 - acc: 0.8227 - val_loss: 0.3844 - val_acc: 0.8567
Epoch 3/10
942/942 [==============================] - 1s 1ms/step - loss: 0.3525 - acc: 0.8450 - val_loss: 0.3772 - val_acc: 0.8567
Epoch 4/10
942/942 [==============================] - 1s 1ms/step - loss: 0.2188 - acc: 0.9289 - val_loss: 0.3994 - val_acc: 0.8500
Test...
              precision    recall  f1-score   support

           0       0.89      0.95      0.92       257
           1       0.46      0.28      0.35        43

    accuracy                           0.85       300
   macro avg       0.67      0.61      

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


942/942 [==============================] - 1s 1ms/step - loss: 1.0406 - acc: 0.6550 - val_loss: 0.4922 - val_acc: 0.8100
Epoch 2/10
942/942 [==============================] - 1s 1ms/step - loss: 0.5147 - acc: 0.7877 - val_loss: 0.4863 - val_acc: 0.8100
Epoch 3/10
942/942 [==============================] - 1s 1ms/step - loss: 0.5074 - acc: 0.7877 - val_loss: 0.4874 - val_acc: 0.8100
Epoch 4/10
942/942 [==============================] - 1s 1ms/step - loss: 0.4945 - acc: 0.7877 - val_loss: 0.4869 - val_acc: 0.8100
Test...
              precision    recall  f1-score   support

           0       0.81      1.00      0.90       243
           1       0.00      0.00      0.00        57

    accuracy                           0.81       300
   macro avg       0.41      0.50      0.45       300
weighted avg       0.66      0.81      0.72       300

Reporting Category:  Diet and Maintenance
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 1s 1ms

#**HAN**

In [0]:
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine.topology import Layer


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            # 1
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
            # next add a Dense layer (for classification/regression) or whatever...
            # 2
            hidden = LSTM(64, return_sequences=True)(words)
            sentence = Attention()(hidden)
            # next add a Dense layer (for classification/regression) or whatever...
        """
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  # e = K.dot(x, self.W)
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim


In [0]:
from keras import Input, Model
from keras.layers import Embedding, Dense, Bidirectional, CuDNNLSTM, TimeDistributed

class HAN(object):
    def __init__(self, maxlen_sentence, maxlen_word, max_features, embedding_dims,
                 class_num=2,
                 last_activation='softmax'):
        self.maxlen_sentence = maxlen_sentence
        self.maxlen_word = maxlen_word
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        # Word part
        input_word = Input(shape=(self.maxlen_word,))
        x_word = Embedding(self.max_features, self.embedding_dims, input_length=self.maxlen_word)(input_word)
        x_word = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x_word)  # LSTM or GRU
        x_word = Attention(self.maxlen_word)(x_word)
        model_word = Model(input_word, x_word)

        # Sentence part
        input = Input(shape=(self.maxlen_sentence, self.maxlen_word))
        x_sentence = TimeDistributed(model_word)(input)
        x_sentence = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x_sentence)  # LSTM or GRU
        x_sentence = Attention(self.maxlen_sentence)(x_sentence)

        output = Dense(self.class_num, activation=self.last_activation)(x_sentence)
        model = Model(inputs=input, outputs=output)
        return model

In [0]:
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = len(vocab)
maxlen_sentence = 16
maxlen_word = 25
batch_size = 32
embedding_dims = 50
epochs = 10

print('Pad sequences (samples x time)...')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen_sentence * maxlen_word)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen_sentence * maxlen_word)
x_train = x_train.reshape((len(x_train), maxlen_sentence, maxlen_word))
x_test = x_test.reshape((len(x_test), maxlen_sentence, maxlen_word))
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)


print('Build model...')
model = HAN(maxlen_sentence, maxlen_word, max_features, embedding_dims).get_model()
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')

for category in categories:
    print('Reporting Category: ', category)

    y_train_label = df_train[category].to_list()
    y_test_label = df_test[category].to_list()

    y_train = to_categorical(y_train_label, num_classes=None)
    y_test = to_categorical(y_test_label, num_classes=None)

    model.fit(x_train,y_train,
              batch_size=batch_size,
              epochs=epochs,
              callbacks=[early_stopping],
              validation_data=(x_test, y_test))
    print('Test...')
    result = model.predict(x_test)
    predictions = [np.argmax(x) for x in result]
    print(classification_report(y_test_label,predictions))

Pad sequences (samples x time)...
x_train shape: (942, 16, 25)
x_test shape: (300, 16, 25)
Build model...
Train...
Reporting Category:  Uncertainity of Post_Diagnosis
Train on 942 samples, validate on 300 samples
Epoch 1/10
942/942 [==============================] - 3s 4ms/step - loss: 0.4840 - acc: 0.8068 - val_loss: 0.4001 - val_acc: 0.8567
Epoch 2/10
942/942 [==============================] - 1s 600us/step - loss: 0.4365 - acc: 0.8217 - val_loss: 0.4043 - val_acc: 0.8567
Epoch 3/10
942/942 [==============================] - 1s 599us/step - loss: 0.3900 - acc: 0.8503 - val_loss: 0.3714 - val_acc: 0.8567
Epoch 4/10
942/942 [==============================] - 1s 621us/step - loss: 0.2751 - acc: 0.8896 - val_loss: 0.6250 - val_acc: 0.8600
Epoch 5/10
942/942 [==============================] - 1s 600us/step - loss: 0.1427 - acc: 0.9490 - val_loss: 0.5545 - val_acc: 0.7700
Epoch 6/10
942/942 [==============================] - 1s 645us/step - loss: 0.0607 - acc: 0.9841 - val_loss: 0.5843 - v

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


942/942 [==============================] - 1s 638us/step - loss: 0.8324 - acc: 0.7113 - val_loss: 0.5111 - val_acc: 0.8100
Epoch 2/10
942/942 [==============================] - 1s 656us/step - loss: 0.5090 - acc: 0.7877 - val_loss: 0.4681 - val_acc: 0.8100
Epoch 3/10
942/942 [==============================] - 1s 627us/step - loss: 0.4902 - acc: 0.7887 - val_loss: 0.4563 - val_acc: 0.8100
Epoch 4/10
942/942 [==============================] - 1s 625us/step - loss: 0.4719 - acc: 0.7909 - val_loss: 0.4570 - val_acc: 0.8267
Epoch 5/10
942/942 [==============================] - 1s 591us/step - loss: 0.4320 - acc: 0.8132 - val_loss: 0.4287 - val_acc: 0.8200
Epoch 6/10
942/942 [==============================] - 1s 616us/step - loss: 0.3451 - acc: 0.8705 - val_loss: 0.4430 - val_acc: 0.8167
Epoch 7/10
942/942 [==============================] - 1s 626us/step - loss: 0.2258 - acc: 0.9268 - val_loss: 0.4783 - val_acc: 0.8167
Test...
              precision    recall  f1-score   support

          